<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png" width="40px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Simple Decision Prompts</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundamental building blocks for eliciting effective responses from AI models. This module demonstrates how to use common prompt engineering techniques within Semantic Kernel. If you’ve used ChatGPT or Microsoft Copilot, you’re already familiar with prompting: given an instruction, a language model predicts the most likely and relevant response. With Semantic Kernel, you can build applications, services, and APIs that execute these prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal is to maximize the quality and clarity of the AI’s output, often by using specific wording, added context, or concrete examples within the prompt.

By combining Decision Intelligence with Prompt Engineering, you can create “Generative AI Decision Intelligence.” This approach leverages GenAI models to reason through complex tasks, gather intelligence, recommend decisions, and communicate results more effectively.

-----
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.8"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.8"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connection info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.8 Microsoft.Extensions.Configuration.Json, 9.0.8

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.65.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.65.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.65.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.65.0

Using Azure OpenAI Service


-----
### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [15]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

General Formatting Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var response = simplePromptResponse.GetValue<string>(); 

// Display the response from the Semantic Kernel as Markdown format
response.DisplayAs("text/markdown");

### Five decision-making frameworks

#### 1) Decision Trees
What it is: A graphical/algorithmic model that maps choices, chance events, outcomes, and payoffs into a branching structure.
How it helps: Forces you to list alternatives, possible events, and consequences explicitly; lets you incorporate probabilities and expected values to compare options quantitatively; clarifies contingent decisions (what you would do after each event). Useful for investments, project selection, medical decisions, and any sequential decisions under uncertainty.

#### 2) Multi‑Criteria Decision Analysis (MCDA) / Analytic Hierarchy Process (AHP)
What it is: A family of methods for evaluating options against multiple, often conflicting criteria by scoring and weighting each criterion.
How it helps: Makes tradeoffs explicit (weights and scores), reduces implicit bias by forcing numeric comparisons, and supports sensitivity analysis to see how changes in priorities affect the outcome. Good for vendor selection, policy choices, hiring/prioritization, and any complex decision where multiple objectives matter.

#### 3) Bayesian Reasoning / Bayesian Decision Theory
What it is: A probabilistic framework that updates beliefs based on new evidence and chooses actions that maximize expected utility given current beliefs.
How it helps: Encourages quantifying uncertainty and updating estimates as information arrives, which improves calibration and prevents stubborn adherence to prior assumptions; it also supports formal tradeoffs between risks and rewards. Especially helpful in diagnostics, forecasting, A/B testing, and adaptive planning where data accumulates over time.

#### 4) OODA Loop (Observe–Orient–Decide–Act)
What it is: An iterative cycle emphasizing rapid observation, contextual orientation, decision, and action, followed by re‑observation.
How it helps: Prioritizes situational awareness and fast feedback, enabling adaptive decisions in dynamic or adversarial environments; helps avoid analysis paralysis by structuring fast, successive refinements. Best for time‑sensitive operations, crisis management, competitive strategy, or real‑time control systems.

#### 5) Premortem / Red‑Teaming
What it is: A foresight and critique approach where a team imagines a future failure (premortem) or takes an adversarial view (red team) to surface weaknesses and hidden assumptions.
How it helps: Reduces overconfidence and groupthink by eliciting failure modes, missing information, and risky assumptions before committing; improves robustness by generating mitigations and alternative plans. Useful in project launches, high‑stakes strategy, policy rollout, and complex system design where blind spots are costly.

-----
### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 **Note:** An average human can read between 25-40 Tokens / second. Therefore, while streaming certainly helps with providing AI output to the user, it begins to lose its effectiveness at large token velocity. Furthermore, certain intermediate and governance (Responsible AI) outputs can become more complex to integrat with streaming.

In [16]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Here are five widely useful decision-making frameworks and how each improves analysis and reasoning in different scenarios.

### Decision Tree Analysis (with Expected Value and sensitivity)
- What it does: Maps choices, chance events, outcomes and payoffs visually; attaches probabilities and values to compute expected values (EV).
- How it helps: Forces explicit structuring of options and uncertain events, makes trade-offs and conditional paths clear, and shows where probabilities or payoffs most affect the decision (sensitivity).
- Best for: Decisions with discrete options and identifiable outcomes (project choices, investments, go/no‑go decisions).

### Multi‑Criteria Decision Analysis (MCDA) / Analytic Hierarchy Process (AHP)
- What it does: Breaks a decision into criteria and alternatives, assigns weights to criteria and scores to alternatives, and aggregates to rank options (AHP uses pairwise comparisons).
- How it helps: Makes subjective preferences explicit and structured, clari

-----
### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can output Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting output. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [4]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework | How it supports better analysis and reasoning |
|---|---|
| Decision Matrix (Weighted Scoring) | Breaks complex choices into criteria, assigns weights and scores to options, produces a quantitative comparison that clarifies trade‑offs, reduces reliance on intuition, and enables sensitivity analysis. |
| Cost–Benefit Analysis (CBA) | Converts costs and benefits into common units (often monetary or utility), compares net present value or return on investment, forces explicit accounting for hidden costs, timing, and externalities. |
| SWOT Analysis | Structures qualitative assessment into Strengths, Weaknesses (internal) and Opportunities, Threats (external), helping identify strategic levers, surface risks and align decisions with context and capabilities. |
| Decision Trees (including probabilistic/expected‑value analysis) | Maps choices, chance events, outcomes and probabilities, enabling explicit modeling of uncertainty, computation of expected values, and visualization of sequential decisions and contingent plans. |
| Bayesian Updating / Bayesian Decision Theory | Uses priors and new evidence to update probabilities and choose actions that maximize expected utility; formalizes uncertainty management, reduces anchoring bias, and integrates data with expert judgment. |

-----
### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Temperature or other settings.

> **📝 Note:** The supported prompt settings are dependent on the API plus the specific model version. For example, an AI model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available. Types of model execution (general versus reasoning) will also have different execution setting parameters. 

In [7]:
PromptExecutionSettings promptExecutionSettings = new();

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}

// Create a new KernelArguments object with the OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(promptExecutionSettings);

var promptResponseWithOpenAI = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat, kernelArguments);
var response = promptResponseWithOpenAI.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework | How it supports better analysis and reasoning |
|---|---|
| OODA Loop (Observe–Orient–Decide–Act) | Structures fast, iterative cycles of sensing and response; improves situational awareness, enables rapid hypothesis testing and course corrections as conditions change. |
| Decision Matrix / Weighted Scoring | Breaks choices into criteria with explicit weights and scores; forces trade-off analysis, reduces bias by making priorities and evaluations transparent and comparable. |
| Cost–Benefit Analysis (CBA) | Converts options into comparable monetary (or utility) terms to quantify trade-offs and opportunity costs; clarifies net value and supports rational resource allocation. |
| Bayesian Reasoning | Treats beliefs as probabilities and updates them with new evidence; makes uncertainty explicit, improves calibration of expectations, and guides decisions under incomplete information. |
| Pre‑mortem Analysis | Imagines future failure to identify hidden assumptions, risks, and failure modes before acting; encourages mitigation planning and reduces overconfidence and blind spots. |

-----
### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decision Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [ ]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

PromptExecutionSettings promptExecutionSettings = new();

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
// Create a new KernelArguments object with the AzureOpenAI / OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(promptExecutionSettings);


var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

-----
### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full **Decision Intelligence** framework will not be used, rather a simple request for Artificial Intelligence to recommend a path forward (recommendation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [10]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommendation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

PromptExecutionSettings promptExecutionSettings = new();

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
// Create a new KernelArguments object with the AzureOpenAI / OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(promptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### Recommendation

| Recommendation | Why (concise rationale) | Key trade-offs (most important) | Concrete next steps (first 12 months) | Contingencies / when to change course |
|---|---:|---|---|---|
| Start at the community college with an intentional, transfer-focused plan to a well-regarded four‑year university after 1–2 years. | Saves substantial money while giving Alex low‑risk space to explore majors (business, psychology, arts) and avoid costly major-switching at the expensive four‑year; preserve ability to transfer later to build the professional network and access internships once she’s more certain. | Pros: Big cost savings (reduces need for large loans); time to clarify major; lower financial risk.  Cons: Delay in full four‑year campus networking and on‑campus internship pipelines; transfer requires planning to avoid credit loss and timeline slip. | 1) Select a target 4‑year (or 2–3) now and obtain articulation/transfer agreement details. 2) Work with CC advisor to choose fully transferable general ed + exploratory courses aligned to possible majors. 3) Aim for a strong GPA (target 3.5+ for competitive transfers) and document coursework. 4) Pursue internships, volunteer, faculty contacts, and CC leadership roles to start building experience and references. 5) Apply for scholarships and external internships; keep family finances and loan needs updated. 6) Visit target campuses in year 1 and meet transfer admissions reps. | If by the end of year 1 Alex is clear about her major and can secure significant scholarships or other funding to offset the 4‑year cost, consider transferring early. If GPA or credits fall behind transfer requirements, extend CC plan to recover (summer courses, tutoring). If a target major is highly competitive, re-evaluate required GPA/portfolio and consider applying to multiple transfer schools or staying an extra semester to strengthen application. |

